In [1]:
import otoole
print(otoole.__version__)
from otoole import read
import os
import sys
sys.path.append('../')

# Suppress FutureWarning messages
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from otoole.utils import (
    _read_file,
)
import xarray as xr

import logging

from feo.osemosys.variables import add_variables
from feo.osemosys.model.constraints.demand import *
from feo.osemosys.model.constraints.capacity_adequacy_a import *
from feo.osemosys.model.constraints.capacity_adequacy_b import *
from feo.osemosys.model.constraints.energy_balance_a import *
from feo.osemosys.model.constraints.energy_balance_b import *
from feo.osemosys.model.constraints.accounting_technology import *
from feo.osemosys.model.constraints.capital_costs import *
from feo.osemosys.model.constraints.operating_costs import *
from feo.osemosys.model.constraints.total_discounted_costs import *
from feo.osemosys.model.constraints.total_capacity import *
from feo.osemosys.model.constraints.new_capacity import *
from feo.osemosys.model.constraints.annual_activity import *
from feo.osemosys.model.constraints.total_activity import *
from feo.osemosys.model.constraints.salvage_value import *
from feo.osemosys.model.constraints.reserve_margin import *
from feo.osemosys.model.constraints.re_targets import *
from feo.osemosys.model.constraints.emissions import *

logger = logging.getLogger(__name__)

1.1.2


In [2]:
model_name = 'simple_model_3_technologies'
config_path = os.path.join('../examples/otoole_config_files', f'{model_name}.yaml')
folder_path = os.path.join('../examples/otoole_csvs', model_name)

print(config_path, folder_path)

../tests/data/model_one.yaml ../tests/data/model_one


In [3]:
with open(config_path, "r") as config_file:
    config = _read_file(config_file, '.yaml')

model, defaults = read(config_path, 'csv', folder_path)

# Reshape the CSV files to create the dataset of sets and parameters
# If the sparse package is available (Python <3.11)
# data_vars = {x: xr.DataArray().from_series(y.VALUE, sparse=True) for x, y in model.items() if config[x]['type'] == 'param'}
data_vars = {x: y.VALUE.to_xarray() for x, y in model.items() if config[x]['type'] == 'param'}
coords = {x: y.values.T[0] for x, y in model.items() if config[x]['type'] == 'set'}
ds = xr.Dataset(data_vars=data_vars, coords=coords)
ds = ds.assign_coords({'_REGION': model['REGION'].values.T[0]})

for param, default in defaults.items():
    if config[param]['type'] == 'param':
        ds[param].attrs['default'] = default
        if default != 0:
            ds[param] = ds[param].fillna(default)
# ds.to_netcdf(f'{model_name}.nc', engine='netcdf4')
# ds = xr.open_dataset(f'{model_name}.nc')
# ds = ds.drop_sel(YEAR=range(2025, 2071))

# Model Creation

In [4]:
# client = Client()
# client

In [5]:
from linopy import Model, solvers, available_solvers

chunks = {
    'YEAR': ds.YEAR.size,
    'TIMESLICE': ds.TIMESLICE.size,
    'REGION': 1,
    'TECHNOLOGY': 100,
    'FUEL': 100,
    'MODE_OF_OPERATION': ds.MODE_OF_OPERATION.size,
    'SEASON': ds.SEASON.size
}

m = Model(force_dim_names=True) #, chunk='auto')

## Variables

In [6]:
m = add_variables(ds, m)

## Discounting

```ampl
param DiscountRate{r in REGION};
param DiscountRateIdv{r in REGION, t in TECHNOLOGY}, default DiscountRate[r];

param DiscountFactor{r in REGION, y in YEAR} :=
	(1 + DiscountRate[r]) ^ (y - min{yy in YEAR} min(yy) + 0.0);
param DiscountFactorMid{r in REGION, y in YEAR} :=
	(1 + DiscountRate[r]) ^ (y - min{yy in YEAR} min(yy) + 0.5);

param OperationalLife{r in REGION, t in TECHNOLOGY};

param CapitalRecoveryFactor{r in REGION, t in TECHNOLOGY} :=
	(1 - (1 + DiscountRateIdv[r,t])^(-1))/(1 - (1 + DiscountRateIdv[r,t])^(-(OperationalLife[r,t])));
param PvAnnuity{r in REGION, t in TECHNOLOGY} :=
	(1 - (1 + DiscountRate[r])^(-(OperationalLife[r,t]))) * (1 + DiscountRate[r]) / DiscountRate[r];

param DiscountRateStorage{r in REGION, s in STORAGE};
param DiscountFactorStorage{r in REGION, s in STORAGE, y in YEAR} :=
	(1 + DiscountRateStorage[r, s]) ^ (y - min{yy in YEAR} min(yy) + 0.0);
param DiscountFactorMidStorage{r in REGION, s in STORAGE, y in YEAR} :=
	(1 + DiscountRateStorage[r, s]) ^ (y - min{yy in YEAR} min(yy) + 0.5);
```

In [7]:
discount_factor = ((1 + ds['DiscountRate']) ** (ds.coords['YEAR'] - min(ds.coords['YEAR'])))
discount_factor_mid = ((1 + ds['DiscountRate']) ** (ds.coords['YEAR'] - min(ds.coords['YEAR']) + 0.5))

discount_factor_idv = ((1 + ds['DiscountRateIdv']) ** (ds.coords['YEAR'] - min(ds.coords['YEAR'])))
discount_factor_mid_idv = ((1 + ds['DiscountRateIdv']) ** (ds.coords['YEAR'] - min(ds.coords['YEAR']) + 0.5))

pv_annuity = (1 - (1 + ds['DiscountRateIdv'])**(-(ds['OperationalLife']))) * (1 + ds['DiscountRateIdv']) / ds['DiscountRateIdv']

capital_recovery_factor = (1 - (1 + ds['DiscountRateIdv'])**(-1))/(1 - (1 + ds['DiscountRateIdv'])**(-(ds['OperationalLife'])))

# Constraints

## Storage

```ampl
s.t. S1_RateOfStorageCharge{r in REGION, s in STORAGE, ls in SEASON, ld in DAYTYPE, lh in DAILYTIMEBRACKET, y in YEAR}:
	sum{t in TECHNOLOGY, m in MODE_OF_OPERATION, l in TIMESLICE: TechnologyToStorage[r,t,s,m] > 0}
	RateOfActivity[r,l,t,m,y] * TechnologyToStorage[r,t,s,m] * Conversionls[l,ls] * Conversionld[l,ld] * Conversionlh[l,lh]
	=
	RateOfStorageCharge[r,s,ls,ld,lh,y];

s.t. S2_RateOfStorageDischarge{r in REGION, s in STORAGE, ls in SEASON, ld in DAYTYPE, lh in DAILYTIMEBRACKET, y in YEAR}:
	sum{t in TECHNOLOGY, m in MODE_OF_OPERATION, l in TIMESLICE: TechnologyFromStorage[r,t,s,m] > 0}
	RateOfActivity[r,l,t,m,y] * TechnologyFromStorage[r,t,s,m] * Conversionls[l,ls] * Conversionld[l,ld] * Conversionlh[l,lh]
	=
	RateOfStorageDischarge[r,s,ls,ld,lh,y];

s.t. S3_NetChargeWithinYear{r in REGION, s in STORAGE, ls in SEASON, ld in DAYTYPE, lh in DAILYTIMEBRACKET, y in YEAR}:
	sum{l in TIMESLICE:Conversionls[l,ls]>0 && Conversionld[l,ld] > 0 && Conversionlh[l,lh] > 0}
	(RateOfStorageCharge[r,s,ls,ld,lh,y] - RateOfStorageDischarge[r,s,ls,ld,lh,y]) * YearSplit[l,y] *
	Conversionls[l,ls] * Conversionld[l,ld] * Conversionlh[l,lh]
	=
	NetChargeWithinYear[r,s,ls,ld,lh,y];

s.t. S4_NetChargeWithinDay{r in REGION, s in STORAGE, ls in SEASON, ld in DAYTYPE, lh in DAILYTIMEBRACKET, y in YEAR}:
	(RateOfStorageCharge[r,s,ls,ld,lh,y] - RateOfStorageDischarge[r,s,ls,ld,lh,y]) * DaySplit[lh,y]
	=
	NetChargeWithinDay[r,s,ls,ld,lh,y];

s.t. S5_and_S6_StorageLevelYearStart{r in REGION, s in STORAGE, y in YEAR}:
	if y = min{yy in YEAR} min(yy)
	then StorageLevelStart[r,s]
	else StorageLevelYearStart[r,s,y-1] + sum{ls in SEASON, ld in DAYTYPE, lh in DAILYTIMEBRACKET}
	NetChargeWithinYear[r,s,ls,ld,lh,y-1]
	=
	StorageLevelYearStart[r,s,y];

s.t. S7_and_S8_StorageLevelYearFinish{r in REGION, s in STORAGE, y in YEAR}:
	if y < max{yy in YEAR} max(yy)
	then StorageLevelYearStart[r,s,y+1]
	else StorageLevelYearStart[r,s,y] + sum{ls in SEASON, ld in DAYTYPE, lh in DAILYTIMEBRACKET}
	NetChargeWithinYear[r,s,ls,ld,lh,y]
	=
	StorageLevelYearFinish[r,s,y];

s.t. S9_and_S10_StorageLevelSeasonStart{r in REGION, s in STORAGE, ls in SEASON, y in YEAR}:
	if ls = min{lsls in SEASON} min(lsls)
	then StorageLevelYearStart[r,s,y]
	else StorageLevelSeasonStart[r,s,ls-1,y] + sum{ld in DAYTYPE, lh in DAILYTIMEBRACKET}
	NetChargeWithinYear[r,s,ls-1,ld,lh,y]
	=
	StorageLevelSeasonStart[r,s,ls,y];

s.t. S11_and_S12_StorageLevelDayTypeStart{r in REGION, s in STORAGE, ls in SEASON, ld in DAYTYPE, y in YEAR}:
	if ld = min{ldld in DAYTYPE} min(ldld)
	then StorageLevelSeasonStart[r,s,ls,y]
	else StorageLevelDayTypeStart[r,s,ls,ld-1,y] + sum{lh in DAILYTIMEBRACKET}
	NetChargeWithinDay[r,s,ls,ld-1,lh,y] * DaysInDayType[ls,ld-1,y]
	=
	StorageLevelDayTypeStart[r,s,ls,ld,y];

s.t. S13_and_S14_and_S15_StorageLevelDayTypeFinish{r in REGION, s in STORAGE, ls in SEASON, ld in DAYTYPE, y in YEAR}:
	if ls = max{lsls in SEASON} max(lsls) && ld = max{ldld in DAYTYPE} max(ldld)
	then StorageLevelYearFinish[r,s,y]
	else if ld = max{ldld in DAYTYPE} max(ldld)
	then StorageLevelSeasonStart[r,s,ls+1,y]
	else StorageLevelDayTypeFinish[r,s,ls,ld+1,y] - sum{lh in DAILYTIMEBRACKET}
	NetChargeWithinDay[r,s,ls,ld+1,lh,y] * DaysInDayType[ls,ld+1,y]
	=
	StorageLevelDayTypeFinish[r,s,ls,ld,y];

#
##########		Storage Constraints				#############
#
s.t. SC1_LowerLimit_BeginningOfDailyTimeBracketOfFirstInstanceOfDayTypeInFirstWeekConstraint{r in REGION, s in STORAGE, ls in SEASON, ld in DAYTYPE, lh in DAILYTIMEBRACKET, y in YEAR}: 0 <= (StorageLevelDayTypeStart[r,s,ls,ld,y]+sum{lhlh in DAILYTIMEBRACKET:lh-lhlh>0} NetChargeWithinDay[r,s,ls,ld,lhlh,y])-StorageLowerLimit[r,s,y];
s.t. SC1_UpperLimit_BeginningOfDailyTimeBracketOfFirstInstanceOfDayTypeInFirstWeekConstraint{r in REGION, s in STORAGE, ls in SEASON, ld in DAYTYPE, lh in DAILYTIMEBRACKET, y in YEAR}: (StorageLevelDayTypeStart[r,s,ls,ld,y]+sum{lhlh in DAILYTIMEBRACKET:lh-lhlh>0} NetChargeWithinDay[r,s,ls,ld,lhlh,y])-StorageUpperLimit[r,s,y] <= 0;
s.t. SC2_LowerLimit_EndOfDailyTimeBracketOfLastInstanceOfDayTypeInFirstWeekConstraint{r in REGION, s in STORAGE, ls in SEASON, ld in DAYTYPE, lh in DAILYTIMEBRACKET, y in YEAR}: 0 <= if ld > min{ldld in DAYTYPE} min(ldld) then (StorageLevelDayTypeStart[r,s,ls,ld,y]-sum{lhlh in DAILYTIMEBRACKET:lh-lhlh<0} NetChargeWithinDay[r,s,ls,ld-1,lhlh,y])-StorageLowerLimit[r,s,y];
s.t. SC2_UpperLimit_EndOfDailyTimeBracketOfLastInstanceOfDayTypeInFirstWeekConstraint{r in REGION, s in STORAGE, ls in SEASON, ld in DAYTYPE, lh in DAILYTIMEBRACKET, y in YEAR}: if ld > min{ldld in DAYTYPE} min(ldld) then (StorageLevelDayTypeStart[r,s,ls,ld,y]-sum{lhlh in DAILYTIMEBRACKET:lh-lhlh<0} NetChargeWithinDay[r,s,ls,ld-1,lhlh,y])-StorageUpperLimit[r,s,y] <= 0;
s.t. SC3_LowerLimit_EndOfDailyTimeBracketOfLastInstanceOfDayTypeInLastWeekConstraint{r in REGION, s in STORAGE, ls in SEASON, ld in DAYTYPE, lh in DAILYTIMEBRACKET, y in YEAR}:  0 <= (StorageLevelDayTypeFinish[r,s,ls,ld,y] - sum{lhlh in DAILYTIMEBRACKET:lh-lhlh<0} NetChargeWithinDay[r,s,ls,ld,lhlh,y])-StorageLowerLimit[r,s,y];
s.t. SC3_UpperLimit_EndOfDailyTimeBracketOfLastInstanceOfDayTypeInLastWeekConstraint{r in REGION, s in STORAGE, ls in SEASON, ld in DAYTYPE, lh in DAILYTIMEBRACKET, y in YEAR}:  (StorageLevelDayTypeFinish[r,s,ls,ld,y] - sum{lhlh in DAILYTIMEBRACKET:lh-lhlh<0} NetChargeWithinDay[r,s,ls,ld,lhlh,y])-StorageUpperLimit[r,s,y] <= 0;
s.t. SC4_LowerLimit_BeginningOfDailyTimeBracketOfFirstInstanceOfDayTypeInLastWeekConstraint{r in REGION, s in STORAGE, ls in SEASON, ld in DAYTYPE, lh in DAILYTIMEBRACKET, y in YEAR}: 	0 <= if ld > min{ldld in DAYTYPE} min(ldld) then (StorageLevelDayTypeFinish[r,s,ls,ld-1,y]+sum{lhlh in DAILYTIMEBRACKET:lh-lhlh>0} NetChargeWithinDay[r,s,ls,ld,lhlh,y])-StorageLowerLimit[r,s,y];
s.t. SC4_UpperLimit_BeginningOfDailyTimeBracketOfFirstInstanceOfDayTypeInLastWeekConstraint{r in REGION, s in STORAGE, ls in SEASON, ld in DAYTYPE, lh in DAILYTIMEBRACKET, y in YEAR}: if ld > min{ldld in DAYTYPE} min(ldld) then (StorageLevelDayTypeFinish[r,s,ls,ld-1,y]+sum{lhlh in DAILYTIMEBRACKET:lh-lhlh>0} NetChargeWithinDay[r,s,ls,ld,lhlh,y])-StorageUpperLimit[r,s,y] <= 0;
s.t. SC5_MaxChargeConstraint{r in REGION, s in STORAGE, ls in SEASON, ld in DAYTYPE, lh in DAILYTIMEBRACKET, y in YEAR}: RateOfStorageCharge[r,s,ls,ld,lh,y] <= StorageMaxChargeRate[r,s];
s.t. SC6_MaxDischargeConstraint{r in REGION, s in STORAGE, ls in SEASON, ld in DAYTYPE, lh in DAILYTIMEBRACKET, y in YEAR}: RateOfStorageDischarge[r,s,ls,ld,lh,y] <= StorageMaxDischargeRate[r,s];
#
#########		Storage Investments				#############
#
s.t. SI1_StorageUpperLimit{r in REGION, s in STORAGE, y in YEAR}: AccumulatedNewStorageCapacity[r,s,y]+ResidualStorageCapacity[r,s,y] = StorageUpperLimit[r,s,y];
s.t. SI2_StorageLowerLimit{r in REGION, s in STORAGE, y in YEAR}: MinStorageCharge[r,s,y]*StorageUpperLimit[r,s,y] = StorageLowerLimit[r,s,y];
s.t. SI3_TotalNewStorage{r in REGION, s in STORAGE, y in YEAR}: sum{yy in YEAR: y-yy < OperationalLifeStorage[r,s] && y-yy>=0} NewStorageCapacity[r,s,yy]=AccumulatedNewStorageCapacity[r,s,y];
s.t. SI4_UndiscountedCapitalInvestmentStorage{r in REGION, s in STORAGE, y in YEAR}: CapitalCostStorage[r,s,y] * NewStorageCapacity[r,s,y] = CapitalInvestmentStorage[r,s,y];
s.t. SI5_DiscountingCapitalInvestmentStorage{r in REGION, s in STORAGE, y in YEAR}: CapitalInvestmentStorage[r,s,y]/(DiscountFactorStorage[r,s,y]) = DiscountedCapitalInvestmentStorage[r,s,y];
s.t. SI6_SalvageValueStorageAtEndOfPeriod1{r in REGION, s in STORAGE, y in YEAR: (y+OperationalLifeStorage[r,s]-1) <= (max{yy in YEAR} max(yy))}: 0 = SalvageValueStorage[r,s,y];
s.t. SI7_SalvageValueStorageAtEndOfPeriod2{r in REGION, s in STORAGE, y in YEAR: (DepreciationMethod[r]=1 && (y+OperationalLifeStorage[r,s]-1) > (max{yy in YEAR} max(yy)) && DiscountRateStorage[r,s]=0) || (DepreciationMethod[r]=2 && (y+OperationalLifeStorage[r,s]-1) > (max{yy in YEAR} max(yy)))}: CapitalInvestmentStorage[r,s,y]*(1-(max{yy in YEAR} max(yy) - y+1)/OperationalLifeStorage[r,s]) = SalvageValueStorage[r,s,y];
s.t. SI8_SalvageValueStorageAtEndOfPeriod3{r in REGION, s in STORAGE, y in YEAR: DepreciationMethod[r]=1 && (y+OperationalLifeStorage[r,s]-1) > (max{yy in YEAR} max(yy)) && DiscountRateStorage[r,s]>0}: CapitalInvestmentStorage[r,s,y]*(1-(((1+DiscountRateStorage[r,s])^(max{yy in YEAR} max(yy) - y+1)-1)/((1+DiscountRateStorage[r,s])^OperationalLifeStorage[r,s]-1))) = SalvageValueStorage[r,s,y];
s.t. SI9_SalvageValueStorageDiscountedToStartYear{r in REGION, s in STORAGE, y in YEAR}: SalvageValueStorage[r,s,y]/((1+DiscountRateStorage[r,s])^(max{yy in YEAR} max(yy)-min{yy in YEAR} min(yy)+1)) = DiscountedSalvageValueStorage[r,s,y];
s.t. SI10_TotalDiscountedCostByStorage{r in REGION, s in STORAGE, y in YEAR}: DiscountedCapitalInvestmentStorage[r,s,y]-DiscountedSalvageValueStorage[r,s,y] = TotalDiscountedStorageCost[r,s,y];
```

## Demand

In [8]:
m = add_demand_constraints(ds, m)

## Capacity Adequacy A

In [9]:
m = add_capacity_adequacy_a_constraints(ds, m)

## Capacity Adequacy B

In [10]:
m = add_capacity_adequacy_b_constraints(ds, m)

## Energy Balance A

In [11]:
m = add_energy_balance_a_constraints(ds, m)

## Energy Balance B

In [12]:
m = add_energy_balance_b_constraints(ds, m)

## Accounting Technology Production/Use

In [13]:
m = add_accounting_technology_constraints(ds, m)

## Capital Costs

In [14]:
m = add_capital_costs_constraints(ds, m, capital_recovery_factor, pv_annuity, discount_factor)

## Salvage Value

In [15]:
m = add_salvage_value_constraints(ds, m)

## Operating Costs

In [16]:
m = add_operating_costs_constraints(ds, m, discount_factor_mid)

## Total Discounted Costs

In [17]:
m = add_total_discounted_costs_constraints(ds, m)

## Total Capacity Constraints

In [18]:
m = add_total_capacity_constraints(ds, m)

## New Capacity Constraints

In [19]:
m = add_new_capacity_constraints(ds, m)

## Annual Activity Constraints

In [20]:
m = add_annual_activity_constraints(ds, m)

## Total Activity Constraints

In [21]:
m = add_total_activity_constraints(ds, m)

## Reserve Margin Constraints

In [22]:
m = add_reserve_margin_constraints(ds, m)

## RE Production Target NTS: Should change demand for production

In [23]:
m = add_re_targets_constraints(ds, m)

In [24]:
m.constraints

linopy.model.Constraints
------------------------
 * EQ_SpecifiedDemand (REGION, TIMESLICE, FUEL, YEAR)
 * CAa1_TotalNewCapacity (REGION, TECHNOLOGY, YEAR)
 * CAa2_TotalAnnualCapacity (REGION, TECHNOLOGY, YEAR)
 * CAa3_TotalActivityOfEachTechnology (REGION, TIMESLICE, TECHNOLOGY, YEAR)
 * CAa4_Constraint_Capacity (REGION, TECHNOLOGY, TIMESLICE, YEAR)
 * CAa5_TotalNewCapacity (REGION, YEAR, TECHNOLOGY)
 * CAb1_PlannedMaintenance (YEAR, REGION, TECHNOLOGY)
 * EBa1_RateOfFuelProduction1 (REGION, FUEL, YEAR, TECHNOLOGY, MODE_OF_OPERATION, TIMESLICE)
 * EBa2_RateOfFuelProduction2 (REGION, TIMESLICE, TECHNOLOGY, FUEL, YEAR)
 * EBa3_RateOfFuelProduction3 (REGION, TIMESLICE, FUEL, YEAR)
 * EBa4_RateOfFuelUse1 (REGION, FUEL, YEAR, TECHNOLOGY, MODE_OF_OPERATION, TIMESLICE)
 * EBa5_RateOfFuelUse2 (REGION, TIMESLICE, TECHNOLOGY, FUEL, YEAR)
 * EBa6_RateOfFuelUse3 (REGION, TIMESLICE, FUEL, YEAR)
 * EBa7_EnergyBalanceEachTS1 (YEAR, TIMESLICE, REGION, FUEL)
 * EBa8_EnergyBalanceEachTS2 (YEAR, TIMESLI

## Emissions

In [25]:
m = add_emissions_constraints(ds, m, discount_factor_mid)

# Objective Function
```ampl
minimize cost: sum{r in REGION, y in YEAR} TotalDiscountedCost[r,y];
```

In [26]:
objective = m['TotalDiscountedCost'].sum(dims=['REGION', 'YEAR'])
m.add_objective(expr=objective, overwrite=True)

# Solving

In [27]:
results_path = f"../results/{model_name}/"
if not os.path.exists(results_path):
    os.makedirs(results_path)

m.to_file(f'../results/{model_name}/{model_name}.lp')

Writing integer variables.: 100%|██████████| 1/1 [00:00<00:00, 910.82it/s]


PosixPath('../results/model_one/model_one.lp')

In [28]:
print(available_solvers)

['glpk', 'cbc']


In [29]:
m.solve(solver_name='glpk', io_api='lp', log_fn=f'../results/{model_name}/solver.log')

Writing integer variables.: 100%|██████████| 1/1 [00:00<00:00, 1098.85it/s]


('ok', 'optimal')

In [30]:
results_path = f"../results/{model_name}/csv/"
if not os.path.exists(results_path):
    os.makedirs(results_path)
    
for variable, data in m.solution.items():
    data = data.to_dataframe(name=variable)
    data[data[variable] != 0.0].to_csv(f"../results/{model_name}/csv/{variable}.csv")